In [1]:
from data import datasets
import albumentations as A
from data import custum_collate
import numpy as np
import glob, cv2
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw
import albumentations.pytorch
import utility
import torch
import torch.distributed as dist
import torch.nn as nn

import torchvision
import os
import random
from yolox.exp import Exp
from yolox.utils import postprocess, crop_mask
from torch.utils.tensorboard import SummaryWriter
import torch.utils.data as data_utils
import datetime
from shutil import copyfile
import time
import modules.evaluation as evaluate

import scipy

In [2]:

w_h = [1280,480] 
transform = A.Compose(
    [
     #A.Resize(width=w_h[0],height=w_h[1]),
     #A.ShiftScaleRotate(p=0.5),
    A.Normalize(mean=(0,0,0),std=(1,1,1)),
    A.pytorch.transforms.ToTensorV2()]
)

In [3]:
save_transform = A.Compose(
    [     
    A.pytorch.transforms.ToTensorV2()],
    
)

In [4]:

def postprocess(prediction, num_classes, conf_thre=0.1, nms_thre=0.85):
    box_corner = prediction.new(prediction.shape)
    box_corner[:, :, 0] = prediction[:, :, 0] - prediction[:, :, 2] / 2
    box_corner[:, :, 1] = prediction[:, :, 1] - prediction[:, :, 3] / 2
    box_corner[:, :, 2] = prediction[:, :, 0] + prediction[:, :, 2] / 2
    box_corner[:, :, 3] = prediction[:, :, 1] + prediction[:, :, 3] / 2
    prediction[:, :, :4] = box_corner[:, :, :4]

    output = [None for _ in range(len(prediction))]
    for i, image_pred in enumerate(prediction):

        # If none are remaining => process next image
        if not image_pred.size(0):
            continue
        # Get score and class with highest confidence
        class_conf, class_pred = torch.max(
            image_pred[:, 5 : 5 + 7], 1, keepdim=True
        )
        loc_conf, loc_pred = torch.max(
            image_pred[:, 5+7 : 5 + 7+9], 1, keepdim=True
        )
        action_conf, action_pred = torch.max(
            image_pred[:, 5+7+9 : 5 + 7 + 9+19], 1, keepdim=True
        )
        reid_feat  = image_pred[:,-256 : ]
        
        action_idx = (image_pred[:, 5+7+9 : 5 + 7 + 9+19]+0.6)//1
        
        conf_mask = (image_pred[:, 4] * class_conf.squeeze()>= conf_thre).squeeze()
        # _, conf_mask = torch.topk((image_pred[:, 4] * class_conf.squeeze()), 1000)
        # Detections ordered as (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
        detections = torch.cat((image_pred[:, :5], class_conf, class_pred.float(),loc_pred.float(),action_idx,reid_feat), 1)
        detections = detections[conf_mask]
        if not detections.size(0):
            continue

        nms_out_index = torchvision.ops.batched_nms(
            detections[:, :4],
            detections[:, 4] * detections[:, 5],
            torch.zeros(detections.shape[0]),
            nms_thre,
        )
        detections = detections[nms_out_index]
        if output[i] is None:
            output[i] = detections
        else:
            output[i] = torch.cat((output[i], detections))

    return output

def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or 2  # line thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=1)

COLORS =[
        [255, 0, 0],     # Red
        [0, 255, 0],     # Green
        [0, 0, 255],     # Blue
        [255, 255, 0],   # Yellow
        [255, 0, 255],   # Magenta
        [0, 255, 255],   # Cyan
        [128, 0, 0],     # Maroon
        [0, 128, 0],     # Green (dark)
        [0, 0, 128],     # Navy
        [128, 128, 0],   # Olive
        [128, 0, 128],   # Purple
        [0, 128, 128],   # Teal
        [255, 165, 0],   # Orange
        [210, 180, 140], # Tan
        [255, 192, 203], # Pink
        [0, 128, 128],   # Teal
        [255, 99, 71],   # Tomato
        [139, 69, 19],   # Saddle Brown
        [0, 128, 0],     # Green (dark)
        [255, 20, 147]   # Deep Pink
    ]
#for _ in range(len(agent_list)):
#    COLORS.append([np.random.randint(0, 255) for _ in range(3)])

In [5]:

def seg_plot_one_box(x,idx, img, mask, color=None, label=None,track_id=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or 2  # line thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    
    c1, c2 = (np.clip(int(x[0]),0,mask.shape[1]), np.clip(int(x[1]),0,mask.shape[0])), (np.clip(int(x[2]),0,mask.shape[1]), np.clip(int(x[3]),0,mask.shape[0]))
    cv2.rectangle(img, c1, c2, color, thickness=1)
    copy_mask = mask[c1[1]:c2[1],c1[0]:c2[0]]==idx
    
    mask_img = np.expand_dims(copy_mask, axis=2)
    mask_img = np.repeat(mask_img, 3, axis=2)
    #print(copy_mask.shape, COLORS[idx%])
    if track_id==None:
        img[c1[1]:c2[1],c1[0]:c2[0],:][copy_mask,:]= img[c1[1]:c2[1],c1[0]:c2[0],:][copy_mask,:]*0.6 + np.array(COLORS[idx%20])*0.4
    else:
        img[c1[1]:c2[1],c1[0]:c2[0],:][copy_mask,:]= img[c1[1]:c2[1],c1[0]:c2[0],:][copy_mask,:]*0.6 + np.array(COLORS[track_id%20])*0.4
        

In [6]:
#Model Create
all_classes =  ['Ped', 'Cyc', 'Mobike', 'Car', 'Bus', 'Cone', 'TL','VehLane', 'OutgoLane', 'MiddleLane', 'IncomLane',  'Pav', 'Jun', 'Xing_L', 'BusStop', 'Parking_L','Red', 'Amber', 'Green', 'MovAway', 'MovTow', 'Blocking', 'Informing', 'Brake', 'Stop', 'IncatLft',
                   'IncatRht', 'HazLit', 'HeadingLft', 'HeadingRht', 'Parking', 'EmVeh', 'School', 'Control', 'Xing']
len_class = len(all_classes)
agent_classes =  ['Ped', 'Cyc', 'Mobike', 'Car', 'Bus', 'Cone', 'TL']
loc_classes =  ['VehLane', 'OutgoLane', 'MiddleLane', 'IncomLane',  'Pav', 'Jun', 'Xing_L', 'BusStop', 'Parking_L']
action_classes =  ['Red', 'Amber', 'Green', 'MovAway', 'MovTow', 'Blocking', 'Informing', 'Brake', 'Stop', 'IncatLft',
                   'IncatRht', 'HazLit', 'HeadingLft', 'HeadingRht', 'Parking', 'EmVeh', 'School', 'Control', 'Xing']
class_nums = [len(agent_classes), len(loc_classes), len(action_classes)]

exp = Exp(class_nums = class_nums)
model = exp.get_model()
#model = load_my_state_dict(model,ckpt["model"])
#ckpt = torch.load('/home/etri/road-dataset/Track_YOLOX/last.ckpt')
#model = utility.load_my_state_dict(model,ckpt["state_dict"])

ckpt = torch.load('./output/Overfit/checkpoints/best.pth.tar', map_location='cpu')
#ckpt = torch.load('./output/2024-01-05-17-29-08_val/checkpoints/last.pth.tar', map_location='cpu')
model = utility.load_my_state_dict(model,ckpt['state_dict'])
model.cuda()

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(128, 64, kernel

In [10]:
import shutil 
import time
try: 
    shutil.rmtree('./Dataset/')
    print("Removed New dir")
except:
    print("Making New dir")
    
#target_folders = ['/home/user/Dataset/RawImg/Ulsan_01_Day/',
#                 '/home/user/Dataset/RawImg/Ulsan_02_Day/']
#target_folders = ['/home/user/Dataset/RawImg/BanseoktoOisam_Snow/',
#                 '/home/user/Dataset/RawImg/CounterClock_Fog/']
#target_folders = ['/home/user/Dataset/RawImg/ETRINear_Fog/',
#                 '/home/user/Dataset/RawImg/ETRItoBanseok_Snow/']
#target_folders = ['/home/user/Dataset/RawImg/Clockwise_Fog/',
#                 '/home/user/Dataset/RawImg/CounterClock2_Fog/']
#target_folders = ['/home/user/Dataset/RawImg/InsideETRI_Day/',
#                 '/home/user/Dataset/RawImg/OisamtoBanseok_Snow/']
#target_folders = ['/home/user/Dataset/RawImg/IllegalParking01_Rain/',
#                 '/home/user/Dataset/RawImg/IllegalParking02_Rain/',
#                 '/home/user/Dataset/RawImg/IllegalParking03_Rain/']
#target_folders = ['/home/user/Dataset/RawImg/IllegalParking04_Rain/',
#                 '/home/user/Dataset/RawImg/IllegalParking05_Rain/']
#target_folders = ['/home/user/Dataset/RawImg/IllegalParking06_Rain/',
#                 '/home/user/Dataset/RawImg/CounterClock_Noisy/']
#target_folders = ['/home/user/Dataset/RawImg/Sejong01_Snow/']
#target_folders = ['/home/user/Dataset/RawImg/Banseok01_Snow/']
#target_folders = ['/home/user/Dataset/RawImg/Banseok02_Snow/',
#                 '/home/user/Dataset/RawImg/KCITY01_Rain/']
#target_folders = ['/home/user/Dataset/RawImg/KCITY01_Snow/',
#                 '/home/user/Dataset/RawImg/KCITY02_Snow/']
#target_folders = ['/home/user/Dataset/RawImg/KCITY03_Snow/',
#                 '/home/user/Dataset/RawImg/KCITY04_Snow/']
#target_folders = ['/home/user/Dataset/RawImg/Hwaseong01_Snow/',
#                 '/home/user/Dataset/RawImg/HwaseongUnstruct02_Snow/']
#target_folders = ['/home/user/Dataset/RawImg/OiSam_Day/']
#target_folders = ['/home/user/Dataset/RawImg/OiSam03_Day/']
#target_folders = ['/home/user/Dataset/RawImg/ETRI_Day_Cons1/',
#                 '/home/user/Dataset/RawImg/ETRI_Day_Cons2/',
#                 '/home/user/Dataset/RawImg/ETRI_Day_Cons3/',
#                 '/home/user/Dataset/RawImg/ETRI_Day_Cons4/',
#                 '/home/user/Dataset/RawImg/ETRI_Day_Cons5/',
#                 '/home/user/Dataset/RawImg/ETRI_Day_Cons6/',
#                 '/home/user/Dataset/RawImg/ETRI_Day_Cons7/']
#target_folders = ['/home/user/Dataset/RawImg/Sejong_L1_00/',
#                 '/home/user/Dataset/RawImg/Sejong_L1_01/']
#target_folders = ['/home/user/Dataset/RawImg/Sejong_L1_04/',
#                 '/home/user/Dataset/RawImg/Sejong_L2_00/']
#target_folders = ['/home/user/Dataset/RawImg/Sejong_L2_02/',
#                 '/home/user/Dataset/RawImg/Sejong_L2_01/']
#target_folders = ['/home/user/Dataset/RawImg/Sejong_L2_04/',
#                 '/home/user/Dataset/RawImg/Sejong_L2_05/']
target_folders = ['/home/user/Dataset/RawImg/Sejong_L3_00/',
                 '/home/user/Dataset/RawImg/Sejong_L3_01/']

for target_folder in target_folders:
    searchLabel = sorted(os.listdir(target_folder))
    new_name = target_folder.split('/')[-2]
    print(new_name)

    filepath = './Dataset/'+new_name+'/img/'
    if not os.path.exists(os.path.dirname(filepath)):
        try:
            os.makedirs(os.path.dirname(filepath))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise


    filepath = './Dataset/'+new_name+'/txt/'
    if not os.path.exists(os.path.dirname(filepath)):
        try:
            os.makedirs(os.path.dirname(filepath))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise


    filepath = './Dataset/'+new_name+'/instance/'
    if not os.path.exists(os.path.dirname(filepath)):
        try:
            os.makedirs(os.path.dirname(filepath))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    count = 0
    frame_num = 0
    t1 = []
    t2 = []
    t3 = []
    t4 = []
    write = True
    model.eval()

    print("Total: ",len(searchLabel))
    with torch.no_grad():
        for jj in range(len(searchLabel)-1):
            if jj%4==0:
                #===============================
                t1.append(time.time())
                #===============================


                img_name = target_folder + '/' + searchLabel[jj]
                img = cv2.imread(img_name)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                clip = transform(image=img)

                images = clip['image'].unsqueeze(0)

                height, width = images.shape[-2:]
                wh = [height, width]
                images = images.cuda(0, non_blocking=True)

                #===============================
                t2.append(time.time())
                #===============================
                output = model(images)
                result,re_mask,bg_vector = output['results'],output['mask_output'],output['bg_vector']
                #===============================
                t3.append(time.time())
                #===============================
                #print(result)
                outputs = postprocess(result, 35, 0.1,0.5)[0]


                #print(max_idx.shape)
                #===============================
                t4.append(time.time())
                #===============================
                clip = save_transform(image=img)
                s_img = clip['image']
                #print(s_img.shape)
                img= s_img.numpy().transpose(1, 2, 0).astype(np.uint8).copy() 

                if outputs==None:
                    continue
                else:
                    
                    path = './Dataset/'+new_name+'/txt/'+searchLabel[jj][:-4]+'.txt'
                    f = open(path, 'w')
                    #print(outputs.shape)
                    outputs = outputs[outputs[:, 0].argsort()]
                    target_vec = torch.cat((bg_vector.unsqueeze(0),outputs[:,-256:]),dim=0)





                    target_mask = re_mask[0,:,:,:]     
                    semseg = torch.einsum("cq,qhw->chw", target_vec, target_mask)            
                    soft_seg = torch.nn.functional.interpolate(torch.nn.functional.softmax(semseg,dim=0).unsqueeze(0),size=(semseg.shape[1]*4,semseg.shape[2]*4),
                                                                     mode='bilinear', align_corners=True).squeeze(0)

                    xyxyn = torch.cat((torch.tensor([[0,0,width,height]]).to(images.device),outputs[:,0:4]) ,dim=0)
                    crop_seg = crop_mask(soft_seg,xyxyn)
                    max_val, max_idx= torch.max(crop_seg,0)

                    
                    outputs = outputs.cpu().numpy()
                    xyxy = outputs[:,0:4]
                    cls = outputs[:,6].astype('int')
                    loc = outputs[:,7].astype('int')
                    action = outputs[:,8:-256].astype('int')
                    action_lab = action
                    
                    #action_lab = np.eye(19)[action.astype('int')]
                    #print(str(action_lab))

                    for i in range(xyxy.shape[0]):
                        #print(i)
                        #str_name = [agent_list[cls[i]],loc_list[loc[i]],action_list[action[i]]]
                        #plot_one_box(xyxy[i], img, label=str_name, color=COLORS[cls[i]])
                        str_name = str(np.int32(xyxy[i][0]+0.5))+' '+str(np.int32(xyxy[i][1]+0.5))+' '+str(np.int32(xyxy[i][2]+0.5))+' '+str(np.int32(xyxy[i][3]+0.5))+' '+str(cls[i])+' '+str(loc[i])+' '+np.array2string(action_lab[i,:].astype('int'),separator=' ')[1:-1]+'\n'
                        f.write(str_name)
                        #print(str_name)
                f.close()
                if write==True:
                    img = img[:, :, ::-1].copy()

                    path = './Dataset/'+new_name+'/img/'+searchLabel[jj][:-4]+'.png'
                    cv2.imwrite(path,img)

                    path = './Dataset/'+new_name+'/instance/'+searchLabel[jj][:-4]+'.png'                
                    cv2.imwrite(path, max_idx.cpu().numpy())
                else:
                    plt.rcParams['figure.figsize'] = [10,10]
                    plt.imshow(max_idx.cpu().numpy())
                    plt.show()   

print("Done")

Removed New dir
Sejong_L3_00
Total:  3457
Sejong_L3_01
Total:  4317
Done


In [8]:
print(img_name)

/home/user/Dataset/RawImg/Sejong_L3_01//00028164.png


In [34]:
import torch

def get_model_parameter_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024 ** 2
    return size_all_mb



In [35]:
from fvcore.nn import FlopCountAnalysis, parameter_count_table

In [36]:

dummy_input = torch.randn(1, 3, 480, 1280)
flop_count = FlopCountAnalysis(model, images)


In [37]:
get_model_parameter_size(model)

303.13541412353516

In [33]:

print(f"FLOPs: {flop_count.total()}")

Unsupported operator aten::silu_ encountered 149 time(s)
Unsupported operator aten::add encountered 22 time(s)
Unsupported operator aten::max_pool2d encountered 3 time(s)
Unsupported operator aten::linalg_vector_norm encountered 5 time(s)
Unsupported operator aten::clamp_min encountered 5 time(s)
Unsupported operator aten::expand_as encountered 5 time(s)
Unsupported operator aten::div encountered 5 time(s)
Unsupported operator aten::sigmoid encountered 12 time(s)
Unsupported operator aten::meshgrid encountered 3 time(s)
Unsupported operator aten::mul encountered 2 time(s)
Unsupported operator aten::exp encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
backbone.AIFI, backbone.AIFI.act, backbone.AIF

FLOPs: 207086780416


In [30]:
print(model(images))

{'results': tensor([[[8.8801e+00, 8.9394e+00, 1.7649e+01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.4702e+00, 9.2728e+00, 1.7800e+01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.1526e+01, 9.2445e+00, 1.7904e+01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [1.1979e+03, 4.5136e+02, 5.9236e+01,  ..., 2.7168e-02,
          0.0000e+00, 0.0000e+00],
         [1.2228e+03, 4.4564e+02, 5.2614e+01,  ..., 6.9387e-03,
          0.0000e+00, 0.0000e+00],
         [1.2621e+03, 4.4831e+02, 3.0487e+01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]]], device='cuda:0',
       grad_fn=<AsStridedBackward0>), 'mask_output': tensor([[[[0.0953, 0.0991, 0.0967,  ..., 0.0943, 0.0893, 0.0920],
          [0.0919, 0.0938, 0.0928,  ..., 0.0981, 0.0965, 0.0906],
          [0.0937, 0.0926, 0.0894,  ..., 0.1032, 0.1056, 0.0988],
          ...,
          [0.0928, 0.0957, 0.0969,  ..., 0.0909, 0.0909, 0.0913],
          [0.0923, 0.0966, 0.

In [25]:
print(images.shape)

torch.Size([1, 3, 480, 1280])
